# What Makes a Movie Successful?

### Jesslyn Himawan (Student #: 65088593) & Rashi Selarka (Student #: 90247693)

## Data Cleaning

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import (
    linear_model, metrics, neural_network, pipeline, model_selection
)

%matplotlib inline

In [2]:
# Loading the data into the notebook

df2 = pd.read_csv("movie_dataset.csv")

In [3]:
df1 = df2[df2['budget'] != 0] # removing budget values that are 0
df = df1[df1['original_language'] == 'en'] # selecting english language films
df = df.drop(['homepage', 'id', 'original_title', 'keywords', 'overview', 'production_companies', 'production_countries', 'spoken_languages', 'status', 'status', 'tagline', 'cast', 'crew', 'director'], axis = 1) # making the dataframe more focused
df = df[df['revenue'] != 0] # removing revenue values that are 0
df['release_date']= pd.to_datetime(df['release_date']) # converting release date to date-time format for plotting
# df.head()

We removed movies that are not in English, and movies with revenue and budget 0.

We also spotted several movies in the dataset with a budget or revenue less than 10,000, which is highly improbable since movies are usually a more expensive affair. For example, in the original dataset, the movie 'Split Second' has a budget of 5 and revenue of 7. After checking the values of budget and revenue for this movie on Wikipedia and IMDB, the values turned out to be 5,000,000 and 7,000,000 respectively.

Therefore, we decided to crosscheck all the values of budget and revenue for movies with a budget or revenue of less than 10,000 in the dataset and update their values with the true ones where necessary.

In [4]:
# identifying entries with budgets under 10000 and replacing them with the real values
df.loc[df["budget"] < 10000]

,index,budget,genres,original_language,popularity,release_date,revenue,runtime,title,vote_average,vote_count
1771,1771,28,Thriller Action Comedy Crime,en,9.670031,2001-12-07,14,93.0,The 51st State,5.9,173
1912,1912,25,Drama,en,4.887940,1999-12-25,13,145.0,Angela's Ashes,7.0,59
2933,2933,11,Drama Action,en,3.375208,1978-04-26,11,145.0,F.I.S.T.,6.4,29
3137,3137,10,Horror Thriller,en,10.013629,2013-09-28,10000000,99.0,Nurse 3-D,4.9,119
3372,3372,7,Thriller Action Horror Science Fiction Crime,en,4.857028,1992-05-01,5,90.0,Split Second,5.7,63
3419,3419,7,Comedy Drama Foreign Romance,en,0.050456,2009-08-09,7,82.0,Bran Nue Dae,5.2,6
3611,3611,4,Drama Romance War,en,1.199451,1932-12-08,25,89.0,A Farewell to Arms,6.2,28
4238,4238,1,Drama Comedy,en,28.276480,1936-02-05,8500000,87.0,Modern Times,8.1,856
4582,4582,218,Documentary Drama,en,1.063439,2003-10-19,1162014,91.0,Tarnation,7.5,22
4608,4608,8,Fantasy Horror Thriller,en,11.818333,1995-09-01,16,98.0,The Prophecy,6.4,138


In [5]:
replace_budget = {28 : 27000000, 25 : 25000000, 11 : 11000000, 10 : 10000000, 7 : 7000000, 4 : 799520, 1 : 1800000, 8 : 8000000, 250 : 250000, 8000 : 187200}

df.replace({"budget": replace_budget}, inplace = True)

In [6]:
df.at[3419,'budget']=6500000 # for the duplicate

In [7]:
# identifying entries with revenue under 10000 and replacing them with the real values
df.loc[df["revenue"] < 10000]

,index,budget,genres,original_language,popularity,release_date,revenue,runtime,title,vote_average,vote_count
1655,1655,23000000,Comedy Romance,en,3.362045,2004-01-09,12,111.0,Chasing Liberty,6.1,152
1771,1771,27000000,Thriller Action Comedy Crime,en,9.670031,2001-12-07,14,93.0,The 51st State,5.9,173
1912,1912,25000000,Drama,en,4.887940,1999-12-25,13,145.0,Angela's Ashes,7.0,59
1999,1999,25000000,Fantasy Adventure Family,en,8.842722,2013-12-05,6399,99.0,The Adventurer: The Curse of the Midas Box,5.1,73
2068,2068,9000000,Comedy Drama,en,20.410354,2007-02-09,46,90.0,Death at a Funeral,6.9,497
2485,2485,16000000,Comedy Drama,en,0.830596,2004-09-03,12,97.0,The Cookout,4.6,10
2874,2874,12000000,Mystery Thriller,en,10.536092,2003-09-09,23,119.0,In the Cut,4.6,89
2933,2933,11000000,Drama Action,en,3.375208,1978-04-26,11,145.0,F.I.S.T.,6.4,29
3121,3121,10000000,Comedy,en,1.728816,2009-10-16,9069,85.0,Janky Promoters,7.0,5
3372,3372,7000000,Thriller Action Horror Science Fiction Crime,en,4.857028,1992-05-01,5,90.0,Split Second,5.7,63


In [8]:
replace_revenue = {12 : 12313323, 14 : 12881605, 13 : 13042112, 6399 : 128471, 46 : 47289148, 23 : 23726793, 11 : 20388920, 5 : 5430822, 25 : 1000000, 7 : 7680, 7202 : 14967, 16 : 16115878, 203 : 206272}

df.replace({"revenue": replace_revenue}, inplace = True)

In [9]:
df.at[3419,'revenue']=6812530 # for the duplicate